In [42]:
import psycopg2
from sql import exec_sql, sql_query
conn = psycopg2.connect(dbname='dvdrental', user='postgres',
                        password='postgres', host='localhost', port=5432)
cur = conn.cursor()

exec_sql(
    """
    DROP TABLE IF EXISTS branches, accounts
    """, conn
)


exec_sql(
    """
    DROP TABLE IF EXISTS color
    """, conn
)



# Создадим таблицы
exec_sql("""CREATE TABLE IF NOT EXISTS color (
    id serial primary key,
    name text,
    hex text -- hex color specification used with html
);

INSERT INTO color (name, hex)
VALUES ('beige', '#F5F5DC'), ('coral', '#FF7F50'),
        ('cyan', '#00FFFF'), ('gold', '#FFD700');
        """, conn)

exec_sql("""
CREATE TABLE IF NOT EXISTS accounts (
    id serial primary key,
    name text,
    branch_name VARCHAR,
    balance DECIMAL
);

INSERT INTO accounts (name, branch_name, balance)
VALUES ('Alice', 'Tomsk', 2000), ('Bob', 'Moscow', 1000),
        ('Igor', 'Tomsk', 1500), ('Oleg', 'Moscow', 3500);

CREATE TABLE IF NOT EXISTS branches(
    id serial primary key,
    name text,
    balance DECIMAL
);

INSERT INTO branches (name, balance)
VALUES ('Moscow', 20000), ('Tomsk', 10000)
""",
         conn);

# Транзакции

Суть транзакции в том, что она объединяет последовательность действий в одну операцию «всё или ничего». Промежуточные состояния внутри последовательности не видны другим транзакциям, и если что-то помешает успешно завершить транзакцию, ни один из результатов этих действий не сохранится в базе данных.

Предположим, что мы хотим перевести 100 долларов со счёта Алисы на счёт Боба. Точное содержание команд здесь не важно, важно лишь то, что для выполнения этой довольно простой операции потребовалось несколько отдельных действий. При этом с точки зрения банка необходимо, чтобы все эти действия выполнились вместе, либо не выполнились совсем. Если Боб получит 100 долларов, но они не будут списаны со счёта Алисы, объяснить это сбоем системы определённо не удастся. И наоборот, Алиса вряд ли будет довольна, если она переведёт деньги, а до Боба они не дойдут. Нам нужна гарантия, что если что-то помешает выполнить операцию до конца, ни одно из действий не оставит следа в базе данных. И мы получаем эту гарантию, объединяя действия в одну транзакцию. Говорят, что транзакция атомарна: с точки зрения других транзакций она либо выполняется и фиксируется полностью, либо не фиксируется совсем.


In [17]:
print(sql_query("""SELECT * FROM accounts"""))
print(sql_query("""SELECT * FROM branches"""))

   id   name branch_name  balance
0   3   Igor       Tomsk   1500.0
1   4   Oleg      Moscow   3500.0
2   1  Alice       Tomsk   1900.0
3   2    Bob      Moscow   1100.0
   id    name  balance
0   2   Tomsk   9000.0
1   1  Moscow  20100.0


In [13]:
exec_sql(
"""
BEGIN;
UPDATE accounts SET balance = balance - 100.00
WHERE name = 'Alice';
UPDATE branches SET balance = balance - 100.00
    WHERE name = (SELECT branch_name FROM accounts WHERE name = 'Alice');
UPDATE accounts SET balance = balance + 100.00
    WHERE name = 'Bob';
UPDATE branches SET balance = balance + 100.00
    WHERE name = (SELECT branch_name FROM accounts WHERE name = 'Bob');
COMMIT;
""", conn)

print(sql_query("""SELECT * FROM accounts"""))
print(sql_query("""SELECT * FROM branches"""))

In [18]:
exec_sql(
"""
BEGIN;
UPDATE accounts SET balance = balance - 100.00
WHERE name = 'Alice';
UPDATE branches SET balance = balance - 100.00
    WHERE name = (SELECT branch_name FROM accounts WHERE name = 'Alice');
UPDATE accounts SET balance = balance + 100.00
    WHERE name = 'Bob';
UPDATE branches SET balance = balance + 100.00
    WHERE name = (SELECT branch_name FROM accounts WHERE name = 'Bob');
ROLLBACK ;
""", conn)

In [19]:
print(sql_query("""SELECT * FROM accounts"""))
print(sql_query("""SELECT * FROM branches"""))

   id   name branch_name  balance
0   3   Igor       Tomsk   1500.0
1   4   Oleg      Moscow   3500.0
2   1  Alice       Tomsk   1900.0
3   2    Bob      Moscow   1100.0
   id    name  balance
0   2   Tomsk   9000.0
1   1  Moscow  20100.0


Операторами в транзакции можно также управлять на более детальном уровне, используя точки сохранения. Точки сохранения позволяют выборочно отменять некоторые части транзакции и фиксировать все остальные. Определив точку сохранения с помощью SAVEPOINT, при необходимости вы можете вернуться к ней с помощью команды ROLLBACK TO. Все изменения в базе данных, произошедшие после точки сохранения и до момента отката, отменяются, но изменения, произведённые ранее, сохраняются.
Отметим, что  ROLLBACK TO — это единственный способ вернуть контроль над блоком транзакций, оказавшимся в прерванном состоянии из-за ошибки системы, не считая возможности полностью отменить её и начать снова.

In [20]:
exec_sql(
    """
BEGIN;
UPDATE accounts SET balance = balance - 100.00
    WHERE name = 'Alice';
SAVEPOINT my_savepoint;
UPDATE accounts SET balance = balance + 100.00
    WHERE name = 'Bob';
-- ошибочное действие... забыть его и использовать счёт Уолли
ROLLBACK TO my_savepoint;
UPDATE accounts SET balance = balance + 100.00
    WHERE name = 'Wally';
COMMIT;
    """, conn
)

In [21]:
print(sql_query("""SELECT * FROM accounts"""))
print(sql_query("""SELECT * FROM branches"""))

   id   name branch_name  balance
0   3   Igor       Tomsk   1500.0
1   4   Oleg      Moscow   3500.0
2   2    Bob      Moscow   1100.0
3   1  Alice       Tomsk   1800.0
   id    name  balance
0   2   Tomsk   9000.0
1   1  Moscow  20100.0


# Изменения таблиц
Существует несколько подходов к изменению таблиц.

1. DROP TABLE. Позволяет удалить таблицу. Обычно используют внутри транзакции, чтобы не нарушить целостность базы.

2. DELETE FROM
- удаление записей в базе или всей таблицы, если оставить остальные поля пустыми

3. UPDATE TABLE:
- изменение значений в таблице (как для всей строки / столбца, так и для отдельных значений)

4. ALTER TABLE:
- переименование самой таблицы
- переименование столбцов
- изменение типов данных
- добавление ограничений
- добавление значений по умолчанию
- итд

In [24]:
exec_sql(
    """
    DROP TABLE IF EXISTS workshop
    """, conn
)

exec_sql("""
CREATE TABLE IF NOT EXISTS workshop (
    id serial primary key,
    name text not null,
    date date,
    beginner boolean default false
);

INSERT INTO workshop (name, date)
VALUES ('Intro to Python', '2017-07-10'),
        ('Python Data Analysis', '2017-08-03'),
        ('Databases', '2017-08-17'),
        ('Intro to R', '2017-09-07');
        """, conn)

## Удаление таблицы (DROP)

In [25]:
exec_sql(
    """
    -- Удаляем таблицу, если такая существует
    DROP TABLE IF EXISTS color
    """, conn
)

## Удалить строки (DELETE)


In [27]:
exec_sql("""
DELETE FROM workshop WHERE id=4;
""", conn)
sql_query("""SELECT * FROM workshop""")

,id,name,date,beginner
0,1,Intro to Python,2017-07-10,False
1,2,Python Data Analysis,2017-08-03,False
2,3,Databases,2017-08-17,False


Отметим, что если вы попытаетесь удалить строки, которые связаны с другими таблицами внешними ключами, то получите ошибку. Чтобы удалить строки в этом случае, нужно либо:
- изменить таблицу при помощи UPDATE (удалить внешний ключ)
- использовать каскадное удаление (CASCADE)

## Изменить значения (UPDATE)

In [32]:
exec_sql(
    """
    UPDATE workshop
    SET name='Introduction to R', beginner=true
    WHERE id=3;
    """, conn
)

In [33]:
sql_query("""SELECT * FROM workshop""")

,id,name,date,beginner
0,1,Intro to Python,2017-07-10,False
1,2,Python Data Analysis,2017-08-03,False
2,3,Introduction to R,2017-08-17,True


In [36]:
exec_sql(
    """
    DROP TABLE IF EXISTS product, product_segment
    """, conn
)

In [37]:
# Используем JOIN
# Создадим две таблицы
from sql import create_table5
exec_sql(
    create_table5, conn
)
print(sql_query("""SELECT * FROM product_segment"""))
print(sql_query("""SELECT * FROM product"""))

   id       segment  discount
0   1  Grand Luxury      0.05
1   2        Luxury      0.06
2   3          Mass      0.10
    id                     name   price net_price  segment_id
0    1                     diam  804.89      None           1
1    2       vestibulum aliquet  228.55      None           3
2    3             lacinia erat  366.45      None           2
3    4  scelerisque quam turpis  145.33      None           3
4    5            justo lacinia  551.77      None           2
5    6     ultrices mattis odio  261.58      None           3
6    7                hendrerit  519.62      None           2
7    8         in hac habitasse  843.31      None           1
8    9           orci eget orci  254.18      None           3
9   10             pellentesque  427.78      None           2
10  11            sit amet nunc  936.29      None           1
11  12           sed vestibulum  910.34      None           1
12  13              turpis eget  208.33      None           3
13  14      

Теперь мы хотим обновить цены в таблице продуктов, используя информацию о скидках из таблицы product_segment.

In [38]:
exec_sql(
    """
UPDATE product
SET net_price = price - price * discount
FROM
product_segment
WHERE
product.segment_id = product_segment.id;
    """, conn)

In [40]:
print(sql_query("""SELECT * FROM product"""))

    id                     name   price  net_price  segment_id
0    1                     diam  804.89     764.65           1
1    2       vestibulum aliquet  228.55     205.70           3
2    3             lacinia erat  366.45     344.46           2
3    4  scelerisque quam turpis  145.33     130.80           3
4    5            justo lacinia  551.77     518.66           2
5    6     ultrices mattis odio  261.58     235.42           3
6    7                hendrerit  519.62     488.44           2
7    8         in hac habitasse  843.31     801.14           1
8    9           orci eget orci  254.18     228.76           3
9   10             pellentesque  427.78     402.11           2
10  11            sit amet nunc  936.29     889.48           1
11  12           sed vestibulum  910.34     864.82           1
12  13              turpis eget  208.33     187.50           3
13  14        cursus vestibulum  985.45     936.18           1
14  15              orci nullam  841.26     799.20     

## Изменить общие параметры таблицы (ALTER)

## Изменение таблицы

Можно изменить таблицу после ее создания:
- переименовать таблицу
- переименовать столбцы
- изменить тип столбцов
- добавить или отменить ограничения
- добавить или изменить значения по умолчанию.

In [43]:
exec_sql(
"""
BEGIN;
ALTER TABLE color
ADD COLUMN IF NOT EXISTS websafe boolean default true;
SELECT * FROM color;
COMMIT;
""",
    conn
)

# Ограничения

В PostgreSQL можно наложить ограничения на колонки. Например, сделать колонки уникальным и ограничить его потенциальными значениями.

In [49]:
# Давайте создадим новую таблицу, где будет пол клиента и его id.
q = """
DROP TABLE IF EXISTS customer_sex;

CREATE TABLE customer_sex
(
    id serial primary key,
    sex VARCHAR(2)
);

INSERT INTO  customer_sex(sex)
VALUES ('F'), ('M'), ('F'), ('R')
;
"""
exec_sql(q, conn)

Как видим, одно из значений пола совсем нереалистичное. Давайте добавим специальный индекс к столбцу пола.

In [53]:
query = """
ALTER TABLE customer_sex
ADD CONSTRAINT unique_sex CHECK ( sex = 'F' OR sex = 'M' )
"""
exec_sql(query, conn)

check constraint "unique_sex" of relation "customer_sex" is violated by some row



In [54]:
q = """
DELETE FROM customer_sex
WHERE sex <> 'M' AND sex <> 'F';

ALTER TABLE customer_sex
ADD CONSTRAINT unique_sex CHECK ( sex = 'F' OR sex = 'M' );
"""
exec_sql(q, conn)

In [55]:
sql_query("""SELECT * FROM customer_sex""")

,id,sex
0,1,F
1,2,M
2,3,F


In [56]:
exec_sql("""INSERT INTO customer_sex(sex)
VALUES ('R') """, conn)

new row for relation "customer_sex" violates check constraint "unique_sex"
DETAIL:  Failing row contains (5, R).



In [18]:
# Теперь создадим таблицу с электронными почтами клиентов
exec_sql("""
CREATE TABLE customer_email
(
id serial,
email VARCHAR(50)
)
""", conn)

In [17]:
# exec_sql("""
# DROP table customer_email
# """, conn)

В реальности у каждого юзера должна быть одна электронная почта. Давайте наложим уникальное ограничение
на столбец email.

In [19]:
exec_sql("""
ALTER TABLE customer_email
ADD CONSTRAINT unique_email UNIQUE(email)
""", conn)

In [20]:
exec_sql("""
INSERT INTO customer_email(email)
VALUES ('1@mail.ru'), ('2@gmail.com'), ('2@gmail.com')
""", conn)

duplicate key value violates unique constraint "unique_email"
DETAIL:  Key (email)=(2@gmail.com) already exists.



PostgreSQL выдал ошибку об уникальности столбца
Ниже посмотрим на список всех ограничений базы dvdrental.

In [22]:
sql_query("""
select pgc.conname as constraint_name,
       ccu.table_schema as table_schema,
       ccu.table_name,
       ccu.column_name
from pg_constraint pgc
join pg_namespace nsp on nsp.oid = pgc.connamespace
join pg_class  cls on pgc.conrelid = cls.oid
left join information_schema.constraint_column_usage ccu
          on pgc.conname = ccu.constraint_name
          and nsp.nspname = ccu.constraint_schema
order by pgc.conname;
""")

,constraint_name,table_schema,table_name,column_name
0,actor_pkey,public,actor,actor_id
1,address_pkey,public,address,address_id
2,category_pkey,public,category,category_id
3,city_pkey,public,city,city_id
4,country_pkey,public,country,country_id
...,...,...,...,...
205,store_address_id_fkey,public,address,address_id
206,store_manager_staff_id_fkey,public,staff,staff_id
207,store_pkey,public,store,store_id
208,unique_email,public,customer_email,email


# Индексы

Индексы нужны для того, чтобы увеличить скорость выполнения запросов.
Однако, это увеличивает место за жестком диске, которое расходуется на создание этих индексов.

In [57]:
# Список всех индексов базы dvdrental
q = """
SELECT
    tablename,
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    schemaname = 'public'
ORDER BY
    tablename,
    indexname;
"""
sql_query(q)

,tablename,indexname,indexdef
0,accounts,accounts_pkey,CREATE UNIQUE INDEX accounts_pkey ON public.ac...
1,actor,actor_pkey,CREATE UNIQUE INDEX actor_pkey ON public.actor...
2,actor,idx_actor_last_name,CREATE INDEX idx_actor_last_name ON public.act...
3,address,address_pkey,CREATE UNIQUE INDEX address_pkey ON public.add...
4,address,idx_fk_city_id,CREATE INDEX idx_fk_city_id ON public.address ...
5,branches,branches_pkey,CREATE UNIQUE INDEX branches_pkey ON public.br...
6,category,category_pkey,CREATE UNIQUE INDEX category_pkey ON public.ca...
7,city,city_pkey,CREATE UNIQUE INDEX city_pkey ON public.city U...
8,city,idx_fk_country_id,CREATE INDEX idx_fk_country_id ON public.city ...
9,color,color_pkey,CREATE UNIQUE INDEX color_pkey ON public.color...


# Ключи

Реляционность базы обеспечивают первичный (primary) и внешний (foreign) ключи.
Они дают пользователю представления о том, как таблицы связаны друг с другом.
Также они защищают от удаления строк, которые связаны с другой таблицей при помощи внешних ключей.
Давайте посмотрим на список индексов, ключей и ограничений базы `dvdrental`.

In [24]:
from p_tqdm import p_map
import pandas as pd
list_all_tables = sql_query("""
SELECT tablename
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
    schemaname != 'information_schema';
""")['tablename'].tolist()

def get_keys_table(table_name):

    data = sql_query(f"""
    select  constraint_name, table_name, column_name
    from information_schema.key_column_usage
    where constraint_catalog=current_catalog and table_name='{table_name}'
    and position_in_unique_constraint notnull;
    """)
    return data
all_keys = list(p_map(get_keys_table, list_all_tables))
all_keys = pd.concat(all_keys)

  0%|          | 0/17 [00:00<?, ?it/s]

In [29]:
exec_sql("""
DELETE FROM payment
WHERE staff_id = 1
""", conn)

## Views

Мы можем делать "вьюхи", которые позволят нам сохранять запросы и выводить результаты.
Рассмотрим на примере ниже.

In [35]:
exec_sql("""
CREATE VIEW  actor_film_names AS
SELECT title, first_name, last_name
FROM actor a
INNER JOIN film_actor fa
ON a.actor_id=fa.actor_id
INNER JOIN film f
ON f.film_id=fa.film_id;
         """, conn)

In [36]:
sql_query("""
SELECT *
FROM actor_film_names
LIMIT 5;
""")

,title,first_name,last_name
0,Chamber Italian,Alec,Wayne
1,Chamber Italian,Henry,Berry
2,Chamber Italian,Rip,Winslet
3,Chamber Italian,Gina,Degeneres
4,Chamber Italian,Adam,Hopper


## Views (материализованные)

In [59]:
exec_sql("""
CREATE MATERIALIZED VIEW  actor_film_names_mat AS

SELECT title, first_name, last_name
FROM actor a
INNER JOIN film_actor fa
ON a.actor_id=fa.actor_id
INNER JOIN film f
ON f.film_id=fa.film_id;
         """, conn)